In [1]:
import numpy as np
from tqdm . auto import tqdm
import collections

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!pip install datasets transformers evaluate torch


In [5]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import evaluate

# Check for GPU availability
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


2024-07-11 06:00:14.349132: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 06:00:14.349244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 06:00:14.494850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Model checkpoint
MODEL_NAME = "distilbert-base-uncased"
# Tokenize the data with specific parameters
MAX_LENGTH = 384
STRIDE = 128

In [7]:
# Download squad dataset từ HuggingFace
DATASET_NAME = "squad_v2"
raw_datasets = load_dataset ( DATASET_NAME )

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
def preprocess_training_examples(examples):
    
    questions = [q.strip() for q in examples["question"]]

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    
    start_positions = []
    end_positions = []

    # offset_mapping
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        sequence_ids = inputs.sequence_ids(i)
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        answer = answers[sample_idx]
        
        if len(answer['text']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)
                
                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)
    
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

In [10]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)


Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [11]:
len(raw_datasets["train"]), len(train_dataset)

(130319, 131754)

In [12]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids

    return inputs


In [13]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,  
    batched=True,                  
    remove_columns=raw_datasets["validation"].column_names, 
)


Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [14]:
len (raw_datasets["validation"]),len(validation_dataset)

(11873, 12134)

In [15]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="distilbert-finetuned-squadv2", 
    evaluation_strategy="no", 
    save_strategy="epoch", 
    learning_rate=2e-5, 
    num_train_epochs=4, 
    weight_decay=0.01, 
    fp16=True, 
    push_to_hub=True)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=train_dataset, 
    eval_dataset=validation_dataset, 
    tokenizer=tokenizer, 
)

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.083300
1000,2.257500
1500,1.974700
2000,1.812600
2500,1.691700
3000,1.659500
3500,1.580400
4000,1.524100
4500,1.499600
5000,1.508800


TrainOutput(global_step=65880, training_loss=0.8880694166413928, metrics={'train_runtime': 12259.5863, 'train_samples_per_second': 42.988, 'train_steps_per_second': 5.374, 'total_flos': 5.164220562484838e+16, 'train_loss': 0.8880694166413928, 'epoch': 4.0})

In [21]:
trainer.push_to_hub(commit_message="Training complete")


CommitInfo(commit_url='https://huggingface.co/FuuToru/distilbert-finetuned-squadv2/commit/00d7a86b8273bf2703f7a3852faf1bf41030169c', commit_message='Training complete', commit_description='', oid='00d7a86b8273bf2703f7a3852faf1bf41030169c', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
metric = evaluate.load("squad_v2")

In [23]:
import collections
import numpy as np
from tqdm import tqdm

N_BEST = 20 
MAX_ANS_LENGTH = 30  

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example['id']
        context = example['context']
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]['offset_mapping']

            start_indexes = np.argsort(start_logit)[-1:-N_BEST-1:-1].tolist()
            end_indexes = np.argsort(end_logit)[-1:-N_BEST-1:-1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if end_index - start_index + 1 > MAX_ANS_LENGTH:
                        continue

                    answer = {
                        'text': context[offsets[start_index][0]:offsets[end_index][1]],
                        'logit_score': start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x['logit_score'])
            answer_dict = {
                'id': example_id,
                'prediction_text': best_answer['text'],
                'no_answer_probability': 1 - best_answer['logit_score']
            }
        else:
            answer_dict = {
                'id': example_id,
                'prediction_text': '',
                'no_answer_probability': 1.0
            }
        predicted_answers.append(answer_dict)

    theoretical_answers = [
        {'id': ex['id'], 'answers': ex['answers']} for ex in examples
    ]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)


In [24]:
predictions, _, _ = trainer.predict(validation_dataset)

start_logits, end_logits = predictions

results = compute_metrics(
    start_logits,
    end_logits,
    validation_dataset,
    raw_datasets["validation"]
)
results


100%|██████████| 11873/11873 [00:33<00:00, 350.41it/s]


{'exact': 54.12279962941127,
 'f1': 57.794453949720435,
 'total': 11873,
 'HasAns_exact': 74.2914979757085,
 'HasAns_f1': 81.64533598937734,
 'HasAns_total': 5928,
 'NoAns_exact': 34.01177460050462,
 'NoAns_f1': 34.01177460050462,
 'NoAns_total': 5945,
 'best_exact': 67.08498273393414,
 'best_exact_thresh': -10.234375,
 'best_f1': 69.12384142896606,
 'best_f1_thresh': -9.93359375}

In [31]:
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'FuuToru/distilbert-finetuned-squadv2'

pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

INPUT_QUESTION = 'What is my name?'
INPUT_CONTEXT = 'My name is AI Vietnam and I live in Vietnam.'

result = pipe(question=INPUT_QUESTION, context=INPUT_CONTEXT)


In [32]:
result

{'score': 0.8425866365432739, 'start': 11, 'end': 21, 'answer': 'AI Vietnam'}